## Proyecto 3. Entrega 2
### Analsisis Data Final
Diego Duarte 22075 - José Marchena 22398 - Andrés Kou 22305 - Esteban Zambrano 22119

In [6]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import scipy.stats as stats
import statsmodels.stats.diagnostic as diag
import statsmodels.api as sm
import os

In [9]:
# Cargar el archivo CSV
csv_path = "DataFinal.csv"
df = pd.read_csv(csv_path)

# Mostrar la información inicial del DataFrame
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 117 entries, 0 to 116
Data columns (total 45 columns):
 #   Column                                                                         Non-Null Count  Dtype  
---  ------                                                                         --------------  -----  
 0   Grupo de Edad                                                                  117 non-null    object 
 1   Año                                                                            117 non-null    int64  
 2   Total VI Victima                                                               117 non-null    float64
 3   Total VI Agresor                                                               117 non-null    float64
 4   VI Victima Hombre                                                              117 non-null    float64
 5   VI Victima Mujer                                                               117 non-null    int64  
 6   VI Agresor Hombre         

,Grupo de Edad,Año,Total VI Victima,Total VI Agresor,VI Victima Hombre,VI Victima Mujer,VI Agresor Hombre,VI Agresor Mujer,VI Urbano,VI Rural,...,Profesionales científicos e intelectuales,Técnicos y profesionales de nivel medio,Personal de apoyo administrativo,Trabajadores de los servicios y vendedores de comercios y mercados,"Agricultores y trabajadores calificados agropecuarios, forestales y pesqueros","Oficiales, operarios y artesanos de artes mecánicas y de otros oficios",Operadores de instalaciones y máquinas y ensambladores,Ocupaciones elementales,Ocupación no especificada,Promedio de Hijos
0,14 o menos,9,251.0,0.0,27.0,194,10.0,33,134,114.0,...,0,0,2,18,2,3,0,2176,1.0,1.04
1,15-19,9,2748.0,992.0,119.0,2629,767.0,225,1809,1622.0,...,345,214,380,657,22,211,186,55917,36.0,1.25
2,20-24,9,5955.0,4385.0,368.0,5587,3805.0,580,5450,3936.0,...,3214,942,2527,1496,42,657,728,89974,225.0,1.94
3,25-29,9,6190.0,6017.0,478.0,5712,5380.0,637,6664,4334.0,...,3197,837,3371,1338,32,680,969,69989,191.0,3.00
4,30-34,9,5154.0,5358.0,438.0,4716,4841.0,517,5732,3731.0,...,2376,431,2136,900,26,549,698,46550,101.0,4.25


In [10]:
# Revisar los tipos de datos de cada columna
print("Tipos de datos antes de la conversión:")
print(df.dtypes)

# Variables que deberían ser categóricas
categoricas_esperadas = ["Grupo de Edad", "Año"]

# Convertir 'Grupo de Edad' y 'Año' a tipo 'category'
for col in categoricas_esperadas:
    df[col] = df[col].astype("category")

print("\nTipos de datos después de la conversión:")
print(df.dtypes)


Tipos de datos antes de la conversión:
Grupo de Edad                                                                     object
Año                                                                                int64
Total VI Victima                                                                 float64
Total VI Agresor                                                                 float64
VI Victima Hombre                                                                float64
VI Victima Mujer                                                                   int64
VI Agresor Hombre                                                                float64
VI Agresor Mujer                                                                   int64
VI Urbano                                                                          int64
VI Rural                                                                         float64
VI Victima Ninguno                                                     

In [11]:
# Identificar columnas que no son numéricas
no_numeric = df.select_dtypes(exclude=[np.number]).columns.tolist()

print("Variables no numéricas encontradas:")
print(no_numeric)

# Verificar si existen variables categóricas que no sean 'Grupo de Edad' o 'Año'
otras_categoricas = [col for col in no_numeric if col not in categoricas_esperadas]

if otras_categoricas:
    print("\nLas siguientes columnas son categóricas pero no deberían serlo:")
    print(otras_categoricas)
else:
    print("\nNo se encontraron variables categóricas adicionales.")


Variables no numéricas encontradas:
['Grupo de Edad', 'Año']

No se encontraron variables categóricas adicionales.


In [12]:
# Descripción general del DataFrame para variables numéricas
print("Descripción estadística de variables numéricas:")
print(df.describe())

# Descripción de las variables categóricas (esperadas)
print("\nDescripción de variables categóricas:")
for col in categoricas_esperadas:
    print(f"\nValor de la columna '{col}':")
    print(df[col].value_counts())


Descripción estadística de variables numéricas:
       Total VI Victima  Total VI Agresor  VI Victima Hombre  \
count        117.000000        117.000000         117.000000   
mean        3463.008547       3350.427350         770.752137   
std         1976.332381       1965.748591        1339.955621   
min            0.000000          0.000000          27.000000   
25%         2251.000000       1965.000000         222.000000   
50%         3070.000000       3191.000000         370.000000   
75%         5441.000000       5079.000000         474.000000   
max         6825.000000       6573.000000        6418.000000   

       VI Victima Mujer  VI Agresor Hombre  VI Agresor Mujer     VI Urbano  \
count        117.000000         117.000000        117.000000    117.000000   
mean        6471.846154        6039.367521        891.820513   7689.521368   
std         9272.480069        9189.584651       1376.008599  11271.130041   
min          124.000000          10.000000          8.000000   

In [14]:
df["Año"] = df["Año"].astype("category")
print(df.dtypes)


Grupo de Edad                                                                    category
Año                                                                              category
Total VI Victima                                                                  float64
Total VI Agresor                                                                  float64
VI Victima Hombre                                                                 float64
VI Victima Mujer                                                                    int64
VI Agresor Hombre                                                                 float64
VI Agresor Mujer                                                                    int64
VI Urbano                                                                           int64
VI Rural                                                                          float64
VI Victima Ninguno                                                                float64
VI Agresor